# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Directory path to my weather data
csvpath = "../output_data/Weather Data.csv"
weather_data = pd.read_csv(csvpath)

# Dropping NaN
weather_df = weather_data.dropna()
weather_df.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,DateTime
0,0,baramati,18.15,74.58,72.536,33.0,0.0,1.93,IN,2020-03-03 19:47:59
1,1,hilo,19.73,-155.09,70.106,88.0,90.0,1.50,US,2020-03-03 19:45:49
2,2,avarua,-21.21,-159.78,84.470,83.0,7.0,1.50,CK,2020-03-03 19:48:00
3,3,kavaratti,10.57,72.64,84.470,66.0,20.0,2.10,IN,2020-03-03 19:48:00
4,4,oranjestad,12.52,-70.03,80.870,69.0,20.0,8.20,AW,2020-03-03 19:45:59


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure Maps
gmaps.configure(api_key=g_key)

# Locations
locations = weather_df[["Latitude", "Longitude"]]
humidity = weather_df["Humidity (%)"].astype(float)

gmap_fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, max_intensity = 50, point_radius = 10, gradient = None)

gmap_fig.add_layer(heat_layer)
gmap_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
weather_ideal_temp = weather_df[(weather_df["Temperature (F)"] > 70) & (weather_df["Temperature (F)"] < 80)]
weather_ideal_speed = weather_ideal_temp[weather_ideal_temp["Wind Speed (mph)"] < 10]
weather_ideal_clouds = weather_ideal_speed[weather_ideal_speed["Cloudiness (%)"] == 0]
weather_ideal_humidity = weather_ideal_clouds[weather_ideal_clouds["Humidity (%)"] <= 50]
ideal_cities = weather_ideal_humidity.reset_index()
ideal_cities = ideal_cities.drop(columns = ['index', 'Unnamed: 0'])
ideal_cities

,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,DateTime
0,baramati,18.15,74.58,72.536,33.0,0.0,1.93,IN,2020-03-03 19:47:59
1,am timan,11.03,20.28,74.210,18.0,0.0,2.13,TD,2020-03-03 19:49:03
2,yulara,-25.24,130.99,79.070,29.0,0.0,3.60,AU,2020-03-03 19:49:11
3,diapaga,12.07,1.79,76.640,15.0,0.0,2.16,BF,2020-03-03 19:49:28
4,presidencia roque saenz pena,-26.79,-60.44,73.256,44.0,0.0,1.70,AR,2020-03-03 19:49:34
5,adrar,20.50,-10.07,70.538,8.0,0.0,4.35,MR,2020-03-03 19:49:52
6,araouane,18.90,-3.53,73.112,7.0,0.0,4.64,ML,2020-03-03 19:50:01
7,ramanuj ganj,23.80,83.70,74.534,49.0,0.0,1.51,IN,2020-03-03 19:50:30
8,viedma,-40.81,-63.00,79.844,38.0,0.0,6.59,AR,2020-03-03 19:46:25
9,calabozo,8.92,-67.43,78.278,31.0,0.0,3.90,VE,2020-03-03 19:51:30


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Base URL https://maps.googleapis.com/maps/api/place/nearbysearch/output?parameters
hotel_df = []

for city in range(len(ideal_cities["City"])):
    
    latitude = ideal_cities.loc[city]["Latitude"]
    longitude = ideal_cities.loc[city]["Longitude"]
    
    city_coordinates = f"{latitude},{longitude}"


    params = {"location": city_coordinates,
              "radius": 5000,
              "type": "hotels",
              "key": g_key}
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_response = requests.get(base_url, params = params).json()
    
    try:
        hotel_df.append(hotel_response["results"][0]["name"])
    except:
        hotel_df.append("Nearest hotel not found")
        
# Dataframe with nearest hotel
ideal_cities['Hotel Name'] = hotel_df
ideal_cities

,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,DateTime,Hotel Name
0,baramati,18.15,74.58,72.536,33.0,0.0,1.93,IN,2020-03-03 19:47:59,Baramati
1,am timan,11.03,20.28,74.210,18.0,0.0,2.13,TD,2020-03-03 19:49:03,Am Timan
2,yulara,-25.24,130.99,79.070,29.0,0.0,3.60,AU,2020-03-03 19:49:11,Petermann
3,diapaga,12.07,1.79,76.640,15.0,0.0,2.16,BF,2020-03-03 19:49:28,Diapaga
4,presidencia roque saenz pena,-26.79,-60.44,73.256,44.0,0.0,1.70,AR,2020-03-03 19:49:34,Presidencia Roque Sáenz Peña
5,adrar,20.50,-10.07,70.538,8.0,0.0,4.35,MR,2020-03-03 19:49:52,Nearest hotel not found
6,araouane,18.90,-3.53,73.112,7.0,0.0,4.64,ML,2020-03-03 19:50:01,Araouane
7,ramanuj ganj,23.80,83.70,74.534,49.0,0.0,1.51,IN,2020-03-03 19:50:30,Ramanujganj
8,viedma,-40.81,-63.00,79.844,38.0,0.0,6.59,AR,2020-03-03 19:46:25,Viedma
9,calabozo,8.92,-67.43,78.278,31.0,0.0,3.90,VE,2020-03-03 19:51:30,Calabozo


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_cities.iterrows()]
locations = ideal_cities[["Latitude", "Longitude"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = [f'City: {row[0]}, Country: {row[1]}, Hotel Name: {row[2]}' for row in ideal_cities[['City','Country','Hotel Name']].values])
gmap_fig.add_layer(markers)
                                                            
# Display Map
gmap_fig

Figure(layout=FigureLayout(height='420px'))